In [4]:
import torch
from torch.utils.data import DataLoader
from models.resnet_clothing_model import ClothingClassifier
from data.dataset import GarmentDataset
from utils.loss import compute_loss
from utils.metrics import accuracy
from data.transformation import CustomResNetTransform
from tqdm import tqdm
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import numpy as np

In [5]:
"""
DATA SET UP
"""
df = pd.read_csv('dataset/list_category_img.txt', delim_whitespace=True, skiprows=1)
selected_labels = [3, 6, 11, 16, 17, 18, 19, 26, 32, 33, 41]
selected_names = ["Blouse", "Cardigan", "Jacket", "Sweater", "Tank", "Tee", "Top", "Jeans", "Shorts", "Skirts", "Dress"]
df = df[df['category_label'].isin(selected_labels)]
df = df.reset_index(drop=True)


label_mapping = {original: new for new, original in enumerate(selected_labels)}
df['category_label'] = df['category_label'].map(label_mapping)

train_val_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=1331,
    stratify=df['category_label']
)
train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.25,
    random_state=1331,
    stratify=train_val_df['category_label']
)

balanced_train_df = train_df.groupby('category_label', group_keys=False).apply(
    lambda x: x.sample(n=1800, random_state=1331)
).reset_index(drop=True)

minority_classes = [2, 6, 7, 3, 1, 9, 4, 8]
transform_strong = CustomResNetTransform(strong_aug=True)
transform_normal = CustomResNetTransform(strong_aug=False)

train_dataset = GarmentDataset(train_df, minority_classes, transform_strong, transform_normal)
balanced_dataset = GarmentDataset(balanced_train_df, minority_classes, transform_strong, transform_normal)
val_dataset = GarmentDataset(val_df, minority_classes, transform_strong, transform_normal)
test_dataset = GarmentDataset(test_df, minority_classes, transform_strong, transform_normal)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
balanced_loader = DataLoader(balanced_dataset, batch_size=64, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=8)

# Get class weights for loss function
class_sample_counts = train_df['category_label'].value_counts().sort_index().tolist()
class_counts_tensor = torch.tensor(class_sample_counts, dtype=torch.float)
class_weights = 1.0 / class_counts_tensor
class_weights = class_weights / class_weights.mean()

/var/folders/nj/y687jr_s3vdc8l49m6_mn7sm0000gn/T/ipykernel_49126/1674405879.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('dataset/list_category_img.txt', delim_whitespace=True, skiprows=1)
/var/folders/nj/y687jr_s3vdc8l49m6_mn7sm0000gn/T/ipykernel_49126/1674405879.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_train_df = train_df.groupby('category_label', group_keys=False).apply(


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

all_preds = []
all_targets = []

NUM_CLASSES = 11
FROZEN_LAYERS = 60
model = ClothingClassifier(num_classes=NUM_CLASSES, num_frozen_resnet_layers=FROZEN_LAYERS, model_type='resnet').to(device)
model.load_state_dict(torch.load('clothing_classifier2-0.pth', map_location=("cuda" if torch.cuda.is_available() else "cpu")))
model.eval()

# Testing Loop: Collect predictions and true labels
with torch.no_grad():
    for images, category_id in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        category_id = category_id.to(device)
        preds = model(images)
        # Get predicted class indices
        predicted = torch.argmax(preds, dim=1)
        # Append predictions and true labels to lists
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(category_id.cpu().numpy())

# Convert lists to numpy arrays for easier computation
all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

# Overall Test Accuracy
total_correct = np.sum(all_preds == all_targets)
total_samples = len(all_targets)
test_accuracy = (total_correct / total_samples) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%\n")

# Compute Per-Class Accuracy
unique_classes = np.unique(all_targets)
per_class_accuracy = {}
for cls in unique_classes:
    cls_idx = (all_targets == cls)
    accuracy_cls = np.mean(all_preds[cls_idx] == all_targets[cls_idx])
    per_class_accuracy[cls] = accuracy_cls

print("Per Class Accuracy:")
for cls, acc in per_class_accuracy.items():
    print(f"Class {cls}: {acc * 100:.2f}%")

# Compute F1 Scores
f1_macro = f1_score(all_targets, all_preds, average='macro')
f1_micro = f1_score(all_targets, all_preds, average='micro')
f1_weighted = f1_score(all_targets, all_preds, average='weighted')

print("\nF1 Scores:")
print(f"Macro F1: {f1_macro:.4f}")
print(f"Micro F1: {f1_micro:.4f}")
print(f"Weighted F1: {f1_weighted:.4f}")

# Optionally, view a detailed classification report:
print("\nClassification Report:")
print(classification_report(all_targets, all_preds, digits=4))



Using device: cpu


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Testing:   0%|          | 1/743 [00:15<3:10:13, 15.38s/it]